# Penguin Species Prediction Challenge

**Multi-class classification challenge**

Your challenge is to develop a machine learning model for classifying the species of penguins living on islands in the [Palmer Archipelago, Antarctica](https://en.wikipedia.org/wiki/Palmer_Archipelago) using features such as their flipper lengths and body masses.

This is a great opportunity to experiment with and learn about a number of core concepts in machine learning, using [pandas](https://pandas.pydata.org/), [seaborn](https://seaborn.pydata.org/) and [scikit-learn](https://scikit-learn.org/stable/index.html).

This Jupyter notebook will guide you through the various general stages involved in machine learning projects &ndash; including data visualisation, data preprocessing, model selection, model training and model evaluation &ndash; in the context of this challenge, and afterwards, you will be able to submit your test set predictions for evaluation on the [DOXA AI](https://doxaai.com/competition/palmer-penguins) platform.

**Before you get started, make sure to [sign up for an account](https://doxaai.com/sign-up) if you do not already have one and [enrol to take part](https://doxaai.com/competition/palmer-penguins) in the challenge.**

**If you have any questions, feel free to ask them in the [DOXA Community Discord server](https://discord.gg/MUvbQ3UYcf).**


## The machine learning workflow


## Installing and importing useful packages


In [ ]:
%pip install numpy pandas matplotlib seaborn scikit-learn
%pip install -U doxa-cli

In [ ]:
import os

import pandas as pd
import seaborn as sns

pd.set_option("display.max_colwidth", None)

## Loading the data


The data for this challenge was originally made available by AM Horst, AP Hill and KB Gorman as an [R package](https://allisonhorst.github.io/palmerpenguins/) under a [CC0 license](https://allisonhorst.github.io/palmerpenguins/LICENSE.html).

We can get started by downloading the data if we do not have it already.

- `data/train.csv` is a CSV (comma-separated values) file containing the full training dataset (including both the features and the target `species` variable)
- `data/test.csv` is a CSV file containing the test set for which your final model will be used to make predictions


In [ ]:
# Download the dataset if we do not already have it
if not os.path.exists("data"):
    os.makedirs("data")

    !curl https://raw.githubusercontent.com/DoxaAI/educational-challenges/main/palmer-penguins/data/train.csv --output data/train.csv
    !curl https://raw.githubusercontent.com/DoxaAI/educational-challenges/main/palmer-penguins/data/test.csv --output data/test.csv

# Load the data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

## Exploring the data

Before we dive into training machine learning models, it is incredibly important to first take the time to properly explore the data available and understand its characteristics. As part of this, we will be looking to identify the following: (i) whether the dataset contains any missing, corrupted values or otherwise invalid values; (ii) whether there may be any large outliers or anomalies in the dataset (e.g. typos); (iii) how the different feature variables are distributed; and (iv) what relationships there are between the different data variables.

This will guide us as to what we should do next and may even give us clues as to which data preprocessing techniques and model types we may wish to use. After all, if the quality of a dataset is low, you may have to spend some time (or a lot of time 😅) [cleaning](https://en.wikipedia.org/wiki/Data_cleansing) it before it can be used.

In this notebook, we will use some simple statistical methods and common visualisation techniques to explore the penguin data we have.


### The training dataset

Let's get started by taking a look at the training dataset, which contains the following data variables:

- `island`: the island the pengiun is from (`Biscoe`, `Dream` or `Torgersen`)
- `bill_length_mm`: the length of the penguin's culmen (i.e. the upper ridge of its bill) in millimetres
- `bill_depth_mm`: the depth of the penguin's culmen in millimetres
- `flipper_length_mm`: the penguin's flipper length in millimetres
- `body_mass_g`: the penguin's body mass in grams
- `sex`: the sex of the penguin (`male` or `female`)
- `year`: the year

<div>
    <img
        alt="The bill length and depth of a penguin"
        src="https://allisonhorst.github.io/palmerpenguins/reference/figures/culmen_depth.png"
        width="500"
    />
</div>

_Artwork by @allison_horst_


In [ ]:
train_df

In [ ]:
# We can use train_df.head(number_of_rows) to get just the first ten rows

train_df.head(10)

So far, so good &ndash; nothing looks particularly outlandish, so we can continue!

As part of our analysis, it can be useful to find out the following things: what columns we have; what their datatypes are; how many entries there are; and whether there are any missing values in our training dataset we might have to handle (e.g. by dropping the rows they are in or imputing the missing values). One way to do this is to use the `info()` method on our training dataframe `train_df`.


In [ ]:
train_df.info()

From this, we can see that our training set has **275 entries** with the **8 columns** outlined earlier, where `island`, `sex` and `species` are categorical variables and the remaining variables are numeric.

We can also look at some statistical properties of the numerical data using the `describe()` method on our training dataframe.


In [ ]:
train_df.describe()

It is also possible to look at this data grouped by the species of penguin using a pandas `groupby()` operation.

**Note**: the resulting dataframe is transposed with `.T` to make it easier to read


In [ ]:
train_df.groupby("species").describe().T

In [ ]:
# [EXERCISE] How would you get this information grouped by both `species` and `sex`?

Pandas also gives us a way to look at the correlations between the numeric variables in our dataset:


In [ ]:
train_df[["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]].corr()

Finally, it also seems from these results that we have a few rows with missing values:


In [ ]:
train_df[train_df.isnull().any(axis=1)]

Fortunately, the missing values are confined to just eleven rows, which we will be able to handle later when we come to preprocess the data.


### Visualising the training data

So far, we have mainly been looking at raw tables of data, but it is often clearer and more intuitive to visualise the data, which we can do using packages such as [matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/), as we will do.

For example, we can plot the number of penguins of each species in the training dataset in the following way:


In [ ]:
sns.countplot(train_df, y="species")

We can also look at how many penguins of each species there were living on each island in a very similar way.


In [ ]:
sns.countplot(train_df, y="island", hue="species")

Likewise, we can see that for the rows where the `sex` is available, the data seems to be fairly evenly balanced across all three species.


In [ ]:
sns.countplot(train_df, y="sex", hue="species")

We can now start to analyse the distribution of each numeric variable, as well as the relationships between them.

One way to visualise this all in one go is to use [pairplot()](https://seaborn.pydata.org/generated/seaborn.pairplot.html) in seaborn:

- On the diagonal, we will show a [kernel density estimation](https://en.wikipedia.org/wiki/Kernel_density_estimation) (KDE) plot to represent how each numeric variable is distributed for each species. We could also do this with a histogram by specifying `diag_kind="hist"`, but with multiple classes, it becomes a bit more difficult to see the different distributions.
- For the other plots, we can generate a scatter graph for each pair of numeric variables, where the points are colour-coded according to the penguin species.


In [ ]:
sns.pairplot(
    train_df,
    vars=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"],
    hue="species",
    plot_kws={"alpha": 0.6},  # make the points slightly transparent to be easier to see
)

Fortunately, it looks like in many cases, the three species form clusters in the data that can be distinguished from each other. This should give us some hope that this is problem to which machine learning might be suited.

If you wanted to view this in a slightly different way, you could also generate bivariate KDE plots for the off-diagonal graphs instead of scatter plots:


In [ ]:
sns.pairplot(
    train_df,
    vars=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"],
    hue="species",
    plot_kws={"alpha": 0.6},
    kind="kde",
)

Since `sns.pairplot` returns the underlying [PairGrid](https://seaborn.pydata.org/generated/seaborn.PairGrid.html), you can customise the plot further, e.g. to show both representations on the off-diagonal graphs.


In [ ]:
p = sns.pairplot(
    train_df,
    vars=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"],
    hue="species",
    plot_kws={"alpha": 0.6},
)
p.map_offdiag(sns.kdeplot, levels=4, color=".2")

Instead of looking at all pairs of numeric variables at the same time, you can also plot histograms for individual variables if you wish.


In [ ]:
sns.histplot(data=train_df, x="flipper_length_mm", hue="species")

In [ ]:
# We can also overlay a KDE plot over the top if we wish
sns.histplot(data=train_df, x="flipper_length_mm", hue="species", kde=True)

You may have also used **box plots** in the past, but another interesting way to visualise data is to use a **violin plot**.


In [ ]:
sns.violinplot(data=train_df, x="bill_length_mm", hue="species")

In [ ]:
# Seaborn has plenty of customisation options for violin plots
sns.violinplot(data=train_df, x="flipper_length_mm", y="species", hue="sex", split=True)

Great! We are starting to build up a picture of what our data looks like and develop an idea of what we might want to do next.


## The test dataset

Before we move onto preprocessing the data, take a quick look at the test dataset for which we will be making predictions.


In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

Fortunately, it looks like there are no missing values in the test set! This should make our lives a lot easier.


## Preprocessing the data

Now it is your turn &ndash; can you preprocess the data to get it ready for training?

When preprocessing and transforming datasets, we generally seek to do the following things where applicable:

- **Handling missing values**

  Most machine learning models cannot be trained on missing values (often encoded as `NULL` or `NaN`), so we need a strategy to deal with them. The simplest such strategy (which we will do) is to ignore the rows that contain missing values; however, sometimes, you can lose a lot of useful data this way. Another simple strategy is that of **univariate imputation** where you replace the missing values with some descriptive statistic computed from the remaining values in that column (e.g. the mean, median or mode) or even a sensible constant value. Some more advanced ways to impute missing values include [multivariate feature imputation](https://scikit-learn.org/1.5/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer) and [nearest neighbours imputation](https://scikit-learn.org/1.5/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer).

  There is a great guide in the [scikit-learn documentation](https://scikit-learn.org/1.5/modules/impute.html) if you want to learn more!

- **Scaling numerical data**

  Many machine learning algorithms (especially those that use gradient descent in optimisation) do not perform as well and may even struggle to converge on a good solution if the input features all have drastically different scales. Two common ways to deal with this are (i) **normalisation** or **min-max scaling** (where data is shifted and rescaled into the range `[0, 1]` typically) and (ii) **standardisation** where the data is shifted to have a mean of zero and rescaled to have unit variance. Some approaches may be more suitable than others, depending on the models you are using and what you want to achieve, but it can often be a good idea to experiment to see what gives you better results!

- **Encoding categorical data**

  We usually need to encode categorical data before we can train a model on it such as by using a [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), which effectively creates a binary column for each discrete category. Another way is to use an [ordinal encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html); however, an ordinal encoding could imply that the different categories are ordered somehow, which might not be at all desirable!

- **Feature engineering**

  Sometimes, it is possible to transform the raw input features you have into a set of features that are more useful in training. This is beyond the scope of this tutorial, but you may wish to read into it!

If you are interested in learning more about data preprocessing, check out the [scikit-learn documentation](https://scikit-learn.org/stable/modules/preprocessing.html) on the subject. Just note that you will want to preprocess the test set in the same way, so if you rescale your training data, you will want to rescale your test data in exactly the same way (with the parameters computed from the _training_ data)!

**Beware of data leakage**: while you must preprocess your training, validation and test data the same way, make sure you do not leak information about the your validation and test data when doing this (e.g. by standardising your data using a mean and standard deviation calculated from all of the data, rather than just the training data), as this may artificially boost the performance of your model.


### Preprocessing the training data

In this example, we will just split our features from our target variable (`species`), remove rows with missing values, rescale the numerical data and one-hot encode the categorical data.


In [ ]:
train_df = train_df.dropna()

# We can also remove year, since it probably won't be too useful!
X_train = train_df.drop(columns=["species", "year"])
y_train = train_df["species"]

X_test = test_df.drop(columns=["year"])

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder


preprocessor = make_column_transformer(
    # [EXERCISE] How should you scale these variables?
    # (
    #     __________(),
    #     ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"],
    # ),
    (
        OneHotEncoder(),
        ["island", "sex"],
    ),
)

## Training and evaluating models

### A little theory

Our objective is to develop a model with the lowest possible **generalisation error** (or **out-of-sample error**), which is a measure of how well our model performs on unseen data. This is important if we ever want to use the models we train for inference in the real world. Of course, we cannot compute this directly &ndash; the data is unseen! &ndash; so we use the **empirical error** from evaluating the model on a test set, which is not used in training at all, as a proxy. This is what we show on the [competition scoreboard](https://doxaai.com/competition/california-housing/scoreboard).

In the ideal scenario, our model will generalise to perform similarly on both the training and test sets; however, this is not always the case. If our model starts to fit to the noise (or residual variation) in our training dataset, rather than the underlying function we are trying to learn (the signal), we end up **overfitting**; and likewise, if the representation of our model is not rich enough to encode that underlying relationship in the data, our model ends up **underfitting**. Both issues cause our model to perform worse when evaluated out-of-sample.

There are a lot of different models available to us (with different suitabilities for encoding different relationships), each with a range of **hyperparameters** we can tune to affect the learning process; however, if we use the performance of our models on the test set as the basis for updating hyperparameters, we stand the risk of leaking information about and overfitting to the test set.

One approach is to further subdivide our training dataset into a training set and a completely separate **validation set**, but training data is precious and in short supply a lot of the time, and we would not want to overfit to that validation set too. An alternative approach here is to perform (**stratified**) **k-fold cross-validation**, where we (randomly) partition the data into `k` different "folds", train `k` models using each fold for validation and the remaining folds for training, and average the results. You can read more about this in the [scikit-learn documentation](https://scikit-learn.org/stable/modules/cross_validation.html).

Putting this all together, we finally just need a strategy for optimising our hyperparameters. One (albeit potentially slow) way is just to perform a **cross-validated grid search** over a grid of hyperparameter values we want to investigate in order to try out a range of different combinations and see what performs well. When we think we have found the model with the best set of hyperparameters, we can retrain it on all of the training data and then evaluate it on the test set! 😎

### Putting everything into practice

We can now try applying what we know by performing a small grid search of our own on a [support vector machine with a linear kernel](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) (or `LinearSVC`) to find the value of the hyperparameter `C` (which controls the strength of regularisation that is applied) that gives the best performing model!


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier


parameter_grid = {
    # You can experiment with parameters here!
    "model__C": [0.5, 1, 2],
}

pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("model", LinearSVC()),
    ]
)

# Since we do all our preprocessing as part of the pipeline we have made, it will be
# repeated every time a model is trained for a particular fold. Why might this be what we want?

classifier = GridSearchCV(
    pipeline,
    param_grid=parameter_grid,
    cv=5,  # the number of folds to use
    refit=True,
    scoring="f1_micro",
)
classifier.fit(X_train, y_train)

print("Best parameters:", classifier.best_params_)
print("Best micro-averaged F1 score:", classifier.best_score_)

Neat! We just used [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to perform a cross-validated grid search and automatically retrain the model on all of the training data! 🥳

But hang on a minute &ndash; what is a "micro-averaged F1 score" that we are using to evaluate our models?

### Evaluation metrics

There are several different metrics we can use to evaluate the performance of a classifier, most notably **accuracy**; however, accuracy can sometimes be misleading when evaluating on an imbalanced dataset, since it does not take the distribution of the classes into account. Here, we have different numbers of each species in the data, so we might want to consider other metrics.

In a binary classification setting, where we have trained a model to classify examples as belonging to either a positive class or a negative class, **true positives** (TP) and **true negatives** (TN) are outcomes that have been correctly classified by our model. Likewise, **false positives** (FN) and **false negatives** (FN) represent outcomes that have been incorrectly classified as positive and negative, respectively.

**Precision** &ndash; also known as the **positive predictive value** (PPV) &ndash; is the proportion of positive predictions that were correctly classified:

\begin{equation*}
\text{precision} = \frac{TP}{TP + FP}
\end{equation*}

**Recall** &ndash; also known as the **true positive rate** (TPR) or in the binary setting, **sensitivity** &ndash; is the proportion of true positive examples that were correctly classified by the model:

\begin{equation*}
\text{recall} = \frac{TP}{TP + FN}
\end{equation*}

An **F<sub>1</sub> score** combines both precision and recall by taking their harmonic mean:

\begin{equation*}
F_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}} = \frac{2 \cdot TP}{2 \cdot TP + FP + FN}
\end{equation*}

Depending on our objectives, this can make it a useful metric for evaluating the performance of a classifier. Note that as we can see from the equation, it takes true positives, false positives and false negatives into account (but not true negatives).

Now, this is interesting for binary classification, but our challenge is that of multi-class classification, so what can we do to extend these metrics for binary classification to the multi-class setting?

- **Macro-averaging** calculates the metric for each class separately (taking a one-versus-rest approach) and then takes an unweighted mean of the results, weighing each class equally.
- **Micro-averaging** aggregates the prediction outcomes across all classes to calculate the metric, weighing each sample equally.

For this challenge, we decided to go with the micro-averaged F<sub>1</sub> score given the imbalanced nature of the problem.

### Plotting a confusion matrix

A **confusion matrix** is a way to visualise a summary of the predictions made by the classifier we trained where the correctly predicted labels appear on the diagonal. Here is how you can generate one:


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(
    y_true=y_train, y_pred=classifier.predict(X_train)
)

## Submitting our predictions to the DOXA AI platform

We are now ready to generate house price predictions for the test set and upload our work to the DOXA AI platform for evaluation!

**Make sure to [enrol to take part](https://doxaai.com/competition/palmer-penguins) in the challenge if you have not already done so.**


In [ ]:
predictions = classifier.predict(test_df.drop(columns=["year"]))

predictions

In [ ]:
# Prepare our submission

os.makedirs("submission", exist_ok=True)

with open("submission/doxa.yaml", "w") as f:
    f.write(
        "competition: palmer-penguins\nenvironment: cpu\nlanguage: python\nentrypoint: run.py\n"
    )

with open("submission/run.py", "w") as f:
    contents = "\\n".join([str(prediction) for prediction in predictions])
    f.write(
        f"""import os
with open(os.environ["DOXA_STREAMS"] + "/out", "w") as f:
    f.write("{contents}")"""
    )

Next, we need to make sure we are logged in:


In [ ]:
!doxa login

Finally, we can submit the predictions for evaluation:


In [ ]:
!doxa upload submission

Wooo! 🥳 You have just submitted your predictions to the platform &ndash; well done! Take a moment to see how well you have done on the [scoreboard](https://doxaai.com/competition/palmer-penguins/scoreboard).


## Possible improvements

Congratulations &ndash; you have made it to the end! We hope you have enjoyed learning about and applying machine learning to this challenge. Hopefully, you can now start experimenting with your own ideas to see how you can improve the performance of your model!

Here are a few questions and ideas to get you started:

1. **Data visualisation**:

- What other visualisations can you make using **pandas**, **matplotlib** and **seaborn**?

2. **Data preprocessing**:

- What other features might you want to use? Is there a way to select them automatically?
- How might you scale the data differently to boost performance?
- Would applying the [PCA algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) help here?

3. **Model selection**:

- What alternatives are there to running a standard grid search? Take a look at [HalvingGridSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV), [HalvingRandomSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV) and [RandomizedSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)!
- Have you ever tried using [BayesSearchCV](https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html) from the [scikit-optimize](https://scikit-optimize.github.io/stable/) package? It searches the space of hyperparameters using Bayesian optimisation and is a drop-in replacement for [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
- What other models could you use for this regression task? Take a look at the [scikit-learn documentation](https://scikit-learn.org/1.5/supervised_learning.html)!
- Could you improve performance by using an [ensemble of different models](https://scikit-learn.org/stable/modules/ensemble.html)?

## Closing remarks

We hope that you have found this to be a useful and enjoyable exercise in exploring and gaining exposure to some fascinating ideas and concepts in machine learning. We look forward to seeing what you build! Do continue the conversation on the [DOXA Community Discord server](https://discord.gg/MUvbQ3UYcf). 😎
